In [17]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce



TARGET = 'итоговый_статус_займа'
ID_COL = 'id'

train = pd.read_csv('../data/shift_ml_2026_train.csv')
test = pd.read_csv('../data/shift_ml_2026_test.csv')

C:\Users\Кирилл\AppData\Local\Temp\ipykernel_11040\3959980358.py:17: DtypeWarning: Columns (0: совокупный_статус_подтверждения_доходов_заемщиков) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../data/shift_ml_2026_train.csv')


In [5]:
#Словарь с рейтингами
# 1) rating_map для 'рейтинг' (буквы)
rating_order = ['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж']
rating_map_main = {r: i for i, r in enumerate(rating_order, start=1)}

# 2) dop_map для 'допрейтинг' (типа 'А1', 'Б3') — учим на TRAIN
def dop_key(x: str):
    x = str(x)
    # ожидаем формат: буква + число (возможны А10 и т.п.)
    letter = x[0]
    num = int(x[1:]) if x[1:].isdigit() else 0
    return (rating_map_main.get(letter, 999), num)

dop_vals = train['допрейтинг'].dropna().astype(str).unique()
dop_vals_sorted = sorted(dop_vals, key=dop_key)
dop_map = {v: i for i, v in enumerate(dop_vals_sorted, start=1)}

In [6]:
def preprocess_base(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    DROP_ALWAYS = [
        'id',
        'коэфф_невыплаченного_сумм_остатка',
        'непогашенная_сумма_из_тела_займов',
        'код_политики',
        'дата_следующей_выплаты',
        'платежный_график',
        'особая_ситуация'
    ]

    df = df.drop(columns=DROP_ALWAYS, errors='ignore')

    # дата -> возраст кредитной истории
    if 'дата_первого_займа' in df.columns:
        dt = pd.to_datetime(df['дата_первого_займа'], format='%m%Y', errors='coerce')
        df['лет_с_первого_займа'] = (pd.Timestamp.today() - dt).dt.days/365
        df['лет_с_первого_займа'] = df['лет_с_первого_займа'].fillna(0)
        df.drop(columns=['дата_первого_займа'], inplace=True)

    # флаги
    if 'сумма_выплат_по_просрочкам' in df.columns:
        df['есть_просрочки'] = (df['сумма_выплат_по_просрочкам'] > 0).astype(int)
    if 'кол-во_взысканий_за_последний_год' in df.columns:
        df['были_взыскания'] = (df['кол-во_взысканий_за_последний_год'] > 0).astype(int)

    # стаж
    if 'стаж' in df.columns:
        df['стаж'] = df['стаж'].astype(str).str.extract(r'(\d+)')[0]
        df['стаж'] = df['стаж'].fillna(0).astype(int)

    # срок займа 
    if 'срок_займа' in df.columns:
        df['срок_займа'] = df['срок_займа'].astype(str).str.extract(r'(\d+)')[0].astype(float)

    if 'пдн' in df.columns:
        df['пдн'] = df['пдн'].clip(lower=0)

        # === Кодирование 'рейтинг' ===
    if 'рейтинг' in df.columns:
        df['рейтинг'] = df['рейтинг'].map(rating_map_main)

    # === Кодирование 'допрейтинг' ===
    if 'допрейтинг' in df.columns:
        df['допрейтинг'] = df['допрейтинг'].astype(str).map(dop_map)
        # если были NaN, после astype(str) они стали 'nan', вернем обратно
        df.loc[df['допрейтинг'].isin([None]) , 'допрейтинг'] = np.nan

    return df

In [7]:
#Бинарное кодирование
def binary_encode_fit_transform(X_train: pd.DataFrame,
                                X_test: pd.DataFrame,
                                binary_cols: list):

    X_train = X_train.copy()
    X_test = X_test.copy()

    binary_cols_exist = [c for c in binary_cols if c in X_train.columns]

    bin_encoder = ce.BinaryEncoder(cols=binary_cols_exist, return_df=True)

    X_train_be = bin_encoder.fit_transform(X_train)
    X_test_be = bin_encoder.transform(X_test)

    return X_train_be, X_test_be, bin_encoder

In [8]:
def onehot_transform(X_train: pd.DataFrame,
                     X_test: pd.DataFrame,
                     onehot_cols: list,
                     top_prof_n: int = 10):

    X_train = X_train.copy()
    X_test = X_test.copy()

    # top-N профессий (fit по train)
    if 'профессия_заемщика' in X_train.columns:
        top_prof = X_train['профессия_заемщика'].value_counts().nlargest(top_prof_n).index

        X_train['профессия_заемщика'] = X_train['профессия_заемщика'].where(
            X_train['профессия_заемщика'].isin(top_prof), 'other'
        )

        if 'профессия_заемщика' in X_test.columns:
            X_test['профессия_заемщика'] = X_test['профессия_заемщика'].where(
                X_test['профессия_заемщика'].isin(top_prof), 'other'
            )

    # one-hot только по выбранным колонкам
    onehot_cols_exist = [c for c in onehot_cols if c in X_train.columns]

    X = pd.get_dummies(X_train, columns=onehot_cols_exist, drop_first=True)
    X_test = pd.get_dummies(X_test, columns=onehot_cols_exist, drop_first=True)

    # выравниваем test под train
    X_test = X_test.reindex(columns=X.columns, fill_value=0)

    # пропуски
    X = X.fillna(0)
    X_test = X_test.fillna(0)

    return X, X_test


In [9]:
X_raw = train.drop(columns=[TARGET], errors='ignore')

X_base = preprocess_base(X_raw)
test_base = preprocess_base(test)

# порог по пропускам: оставляем колонки, где заполнено >= 70%
MISSING_THRESHOLD = 0.7

# выкидываем фиксированные
X_base = X_base.drop(columns=DROP_ALWAYS, errors='ignore')
test_base = test_base.drop(columns=DROP_ALWAYS, errors='ignore')

# оставляем только колонки, где в TRAIN заполнено >= 70%
keep_cols = X_base.columns[X_base.notna().mean() >= MISSING_THRESHOLD].tolist()

X_base = X_base[keep_cols].copy()
test_base = test_base.reindex(columns=keep_cols) # чтобы столбцы были те же

print("train cols:", X_base.shape[1], "test cols:", test_base.shape[1])

train cols: 79 test cols: 79


In [10]:
binary_cols = [
    'юридический_статус', 'первоначальный_статус_займа',
    'пени_за_дефолт', 'тип_займа', 'тип_предоставления_кредита'
]

onehot_cols = [
    'владение_жильем', 'подтвержден_ли_доход', 'цель_займа', 'регион',
    'пос_стоп_фактор', 'совокупный_статус_подтверждения_доходов_заемщиков',
    'профессия_заемщика'
]

X_base_be, test_base_be, bin_encoder = binary_encode_fit_transform(X_base, test_base, binary_cols)

X, X_test = onehot_transform(X_base_be, test_base_be, onehot_cols, top_prof_n=10)

print(X.shape, X_test.shape)

(1210779, 159) (134531, 159)


In [11]:
assert list(X.columns) == list(X_test.columns)

In [12]:
y = train[TARGET].astype(int)

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [13]:
hgb = HistGradientBoostingClassifier(
    max_depth=6,
    learning_rate=0.05,
    max_iter=300,
    random_state=42
)

hgb.fit(X_train, y_train)

pred_hgb = hgb.predict_proba(X_valid)[:, 1]
roc_auc_hgb = roc_auc_score(y_valid, pred_hgb)

print("HGB ROC-AUC:", roc_auc_hgb)


HGB ROC-AUC: 0.7572451237728649


In [14]:
tree = DecisionTreeClassifier(
    max_depth=6,
    min_samples_leaf=200,
    class_weight='balanced',
    random_state=42
)

tree.fit(X_train, y_train)

pred_tree = tree.predict_proba(X_valid)[:, 1]
roc_auc_tree = roc_auc_score(y_valid, pred_tree)

print("Decision Tree ROC-AUC:", roc_auc_tree)

Decision Tree ROC-AUC: 0.7273771727064544


In [15]:
rf = RandomForestClassifier(
    n_estimators=150,
    max_depth=12,
    min_samples_leaf=100,
    n_jobs=-1,
    random_state=42,
    class_weight='balanced'
)

rf.fit(X_train, y_train)

pred_rf = rf.predict_proba(X_valid)[:, 1]
roc_auc_rf = roc_auc_score(y_valid, pred_rf)

print("Random Forest ROC-AUC:", roc_auc_rf)


Random Forest ROC-AUC: 0.7431234631628371


In [16]:
results = []

results.append(("HistGradientBoosting", roc_auc_hgb))
results.append(("DecisionTree", roc_auc_tree))
results.append(('RandomForest', roc_auc_rf))

comparison = pd.DataFrame(results, columns=["Model", "ROC-AUC"]) \
    .sort_values("ROC-AUC", ascending=False)

comparison

,Model,ROC-AUC
0,HistGradientBoosting,0.757245
2,RandomForest,0.743123
1,DecisionTree,0.727377


Градиентный бустинг показал более высокое качество по метрике ROC-AUC, поэтому оставляем его в качестве основной модели, что ожидаемо, т.к. бустинг уменьшает bias и variance по сравнению с одиночным деревом решений и более эффективно снижает эти же показатели, чем RF

По сравнению с деревом Случайный лес выдает чуть лучше показатель метрики, но обучается вдвое дольше

Дерево решений < Случайный лес < Бустинг